# Talks markdown generator
Takes a TSV of talks with metadata and converts them for use with academicpages.github.io.

TODO: Make this work with BibTex or other databases of talks, rather than Stuart's TSV format.


In [1]:
!pip install getorg --upgrade
import pandas as pd
import os

Requirement already up-to-date: getorg in /srv/paws/lib/python3.4/site-packages
Requirement already up-to-date: geopy in /srv/paws/lib/python3.4/site-packages (from getorg)
Requirement already up-to-date: retrying in /srv/paws/lib/python3.4/site-packages (from getorg)
Requirement already up-to-date: pygithub in /srv/paws/lib/python3.4/site-packages (from getorg)
Requirement already up-to-date: six>=1.7.0 in /srv/paws/lib/python3.4/site-packages (from retrying->getorg)
Requirement already up-to-date: python-jose in /srv/paws/lib/python3.4/site-packages (from pygithub->getorg)
Requirement already up-to-date: pycrypto<2.7.0,>=2.6.0 in /srv/paws/lib/python3.4/site-packages (from python-jose->pygithub->getorg)
Requirement already up-to-date: ecdsa<1.0 in /srv/paws/lib/python3.4/site-packages (from python-jose->pygithub->getorg)
Requirement already up-to-date: future<1.0 in /srv/paws/lib/python3.4/site-packages (from python-jose->pygithub->getorg)


In [2]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,date,talk_type,title,venue,institution,geoloc,url,slug,summary,description
0,2008-03-01,Talk,A Communicative Ethnography of Argumentative S...,Exploring New Media Worlds,Texas A&M University,"College Station, TX",NaN,tamu-communicative-ethnography,NaN,NaN
1,2008-07-19,Talk,Conceptions and Misconceptions Academics Hold ...,Annual Wikimedia Conference (Wikimania),Wikimedia Foundation,"Alexandria, Egypt",NaN,wikimania-academics-wikipedia,NaN,NaN
2,2009-03-28,Talk,Working With/in Wikipedia: Infrastructures of ...,Annual Conference on Science and Technology in...,AAAS,"Wasthington, Dc",NaN,aaas-wikipedia-infrastructures,NaN,NaN
3,2009-04-25,Talk,Evolving Governance and Media Use in Wikipedia...,Media in Transition 6,MIT,"Cambridge, MA",NaN,mit6-media-use-wikipedia,NaN,NaN
4,2009-07-26,Talk,Algorithmic Governance: The Social Roles of Bo...,First Annual Wikiconference NYC,Wikimedia Foundation,"New York, NY",NaN,wikiconf-nyc-algorithmc-governance,NaN,NaN
5,2009-09-25,Talk,Trace Ethnography: An ANT Method for the Study...,the Second Annual Media Sociology Forum,New York University,"New York, NY",NaN,nyu-trace-ethnography,NaN,NaN
6,2009-10-27,Conference proceedings talk,The Social Roles of Bots and Assisted Editing ...,International Symposium on Wikis and Open Coll...,ACM,"Orlando, Florida",http://www.stuartgeiger.com/papers/geiger-wiki...,wikisym-social-roles-bots,A short paper showing the recent explosive gro...,A short paper showing the recent explosive gro...
7,2009-10-28,Talk,Where Are the Missing Wikipedians? The Sociolo...,Annual Meeting of the Society for the Social S...,NaN,"Arlington, Virginia",NaN,4s-missing-wikipedians,NaN,NaN
8,2010-01-10,Talk,The Wisdom of Bots: A Critique of ‘Self-Organi...,Critical Point of View: Wikipedia and the Poli...,Centre for Internet and Society,"Bangalore, India",NaN,cpov-wisdom-of-bots,NaN,NaN
9,2010-02-25,Conference proceedings talk,The Work of Sustaining Order in Wikipedia: The...,Conference on Computer Supported Cooperative Work,ACM,"Savannah, Georgia",http://www.stuartgeiger.com/papers/cscw-sustai...,cscw-banning-vandal,This paper traces out a heterogeneous network ...,This paper traces out a heterogeneous network ...


In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;",
    ">": "&gt;",
    "<": "&lt;",
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [4]:
!mkdir _talks

In [5]:
loc_dict = {}
count = 0
base_url = "staeiou.github.io"

for row, item in talks.iterrows():
    md_filename = str(item.date) + "-" + item.slug + ".md"
    html_filename = str(item.date) + "-" + item.slug 
    year = item.date[:4]
    count += 1
    
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    if item.talk_type is not None:
        md += 'type: "' + item.talk_type + '"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if item.venue is not None:
        md += 'venue: "' + item.venue + '"\n'
        
    if item.date is not None:
        md += "date: " + str(item.date) + "\n"
    
    if item.geoloc is not None:
        md += 'location: "' + str(item.geoloc) + '"\n'
       
    
    if len(str(item.summary))>10:
        md += 'excerpt: "'
        md += html_escape(item.summary) + '"\n'
    elif len(str(item.description))>10:
        if len(str(item.description))>200:
            md += 'excerpt: "'
            md += html_escape(item.description[:200])
            md += '..."\n'
        else:
            md += 'excerpt: "'
            md += html_escape(item.description) + '"\n'
    
    md += "---\n"
    
    if isinstance(item.url, str):
        md += "\n<a href='" + str(item.url) + "'>Link to more information</a>\n" 
    
    if len(str(item.description))>10:
        md += "\n" + html_escape(item.description) + "\n"
    md_filename = os.path.basename(md_filename)
    print(md)
    
    with open("_talks/" + md_filename, 'w') as f:
        f.write(md)

---
title: "A Communicative Ethnography of Argumentative Strategies in a Wikipedian Content Dispute"
collection: talks
type: "Talk"
permalink: /talks/2008-03-01-tamu-communicative-ethnography
venue: "Exploring New Media Worlds"
date: 2008-03-01
location: "College Station, TX"
---

---
title: "Conceptions and Misconceptions Academics Hold About Wikipedia"
collection: talks
type: "Talk"
permalink: /talks/2008-07-19-wikimania-academics-wikipedia
venue: "Annual Wikimedia Conference (Wikimania)"
date: 2008-07-19
location: "Alexandria, Egypt"
---

---
title: "Working With/in Wikipedia: Infrastructures of Knowing and Knowledge Production"
collection: talks
type: "Talk"
permalink: /talks/2009-03-28-aaas-wikipedia-infrastructures
venue: "Annual Conference on Science and Technology in Society"
date: 2009-03-28
location: "Wasthington, Dc"
---

---
title: "Evolving Governance and Media Use in Wikipedia: A Historical Account"
collection: talks
type: "Talk"
permalink: /talks/2009-04-25-mit6-media-us

In [8]:
!ls _talks

2008-03-01-tamu-communicative-ethnography.md
2008-07-19-wikimania-academics-wikipedia.md
2009-03-28-aaas-wikipedia-infrastructures.md
2009-04-25-mit6-media-use-wikipedia.md
2009-07-26-wikiconf-nyc-algorithmc-governance.md
2009-09-25-nyu-trace-ethnography.md
2009-10-27-wikisym-social-roles-bots.md
2009-10-28-4s-missing-wikipedians.md
2010-01-10-cpov-wisdom-of-bots.md
2010-02-25-cscw-banning-vandal.md
2010-03-26-cpov-bot-politics.md
2010-07-10-wikimania-academic-researchers.md
2011-01-03-hicss-trace-ethnography.md
2011-03-04-dml-bots-governance.md
2011-10-05-wikisym-article-deletion.md
2011-11-02-4s-internet-is-here.md
2011-11-03-4s-wikipedian-governance.md
2012-03-31-gcoe-wikipedia-notifications.md
2012-05-02-altchi-ipoxp.md
2012-05-07-chi-fail-whales.md
2012-06-05-icwsm-socialization-wikipedia.md
2012-10-12-infosocial-trace-literacy.md
2012-10-17-4s-time-to-degree.md
2012-10-29-wikisym-methods.md
2013-02-07-203-actor-network-theory.md
2013-02-23-cscw-edit-sessions.md
2013-02-26-cscw-co

In [7]:
!head _talks/2008-03-01-tamu-communicative-ethnography.md

---
title: "A Communicative Ethnography of Argumentative Strategies in a Wikipedian Content Dispute"
collection: talks
type: "Talk"
permalink: /talks/2008-03-01-tamu-communicative-ethnography
venue: "Exploring New Media Worlds"
date: 2008-03-01
location: "College Station, TX"
---
